# Wykład 12 - Wielowątkowość - Podstawy

## 12.0 Model pamięci - podstawy

<img src="mem.png" width="450"/>

<img src="mem2.png" width="450"/>

In [ ]:
package Memory;

public class MyRunnable implements Runnable {

    private int count = 0;

    @Override
    public void run() {
        for(int i = 0; i < 10000; i++)
            this.count++;
        System.out.println(Thread.currentThread().getName() + " : " + this.count);
    }
}


In [ ]:
package Memory;

public class TwoSeparateObjects {

    public static void main(String[] args) {
        Runnable runnable1 = new MyRunnable();
        Runnable runnable2 = new MyRunnable();

        Thread thread1 = new Thread(runnable1, "Thread1");
        Thread thread2 = new Thread(runnable2, "Thread2");

        thread1.start();
        thread2.start();

    }
}


In [ ]:
package Memory;

public class SharedObject {
    public static void main(String[] args) {
        Runnable runnable = new MyRunnable();

        Thread thread1 = new Thread(runnable, "Thread1");
        Thread thread2 = new Thread(runnable, "Thread2");

        thread1.start();
        thread2.start();
    }
}


<img src="cpu.png" width="450"/>

## 12.1 `synchronized`

In [ ]:
public class SynchronizedClass {
    protected Object object = null;
    
    // tylko jeden wątek może wykonać metodę synchronized na raz
    // na instancji klasy
    public synchronized void setObject(Object o){
        this.object = o;
    }
    
    public synchronized Object getObject(){
        return this.object;
    }
    
    public void setObj(Object o){
        // tylko jeden wątek może wykonać blok synchronized na raz
        // na instancji klasy
        synchronized(this){
            this.object = o;
        }
    }
    
    public Object getObj(){
        synchronized(this){
            return this.object;
        }
    }
}

![Synchronized](synchClass.png)

Jeżeli `Thread1` aktualnie wykonuje metodę `setObject`, `Thread2` nie może wykonać `getObject`.

![Synchronized](synchClassinstances.png)

In [ ]:
public class StaticSynchronizedClass {
    protected static Object object = null;

    // tylko jeden wątek może wykonać metodę synchronized na raz
    public static synchronized void setObject(Object o){
        object = o;
    }

    public static synchronized Object getObject(){
        return object;
    }

    public static void setObj(Object o){
        // tylko jeden wątek może wykonać blok synchronized na raz
        synchronized(StaticSynchronizedClass.class){
            this.object = o;
        }
    }

    public static Object getObj(){
        synchronized(StaticSynchronizedClass.class){
            return this.object;
        }
    }
}

![Synchronized](staticSynchClass.png)

Ograniczenia
- tylko jeden wątek może wykonywać metodę lub blok `synchronized`
- brak kontroli nad wątkami oczekującymi

## 12.2 Concurrency vs Parallelism

- Concurrency
- Parallel Execution
- Parallel Concurrent Execution
- Parallelism

### Concurency

![Synchronized](conc.png)

### Parallel Execution

![Synchronized](parExec.png)

### Parallel Concurrent Execution

![Synchronized](parConcExec.png)

### Parallelism

![Synchronized](par.png)

### AsyncTask

![Synchronized](async.png)

## 12.3 `ThreadPool`

![threadPool](threadPool.png)

In [ ]:
[Źródło](https://jenkov.com/tutorials/java-concurrency/thread-pools.html)

import java.util.ArrayList;
import java.util.List;
import java.util.concurrent.ArrayBlockingQueue;
import java.util.concurrent.BlockingQueue;

public class MyThreadPool {

    private final BlockingQueue<Runnable> taskQueue;
    private final List<PoolThreadRunnable> runnables = new ArrayList<>();
    private boolean isStopped = false;

    public MyThreadPool(int noOfThreads, int maxNoOfTasks) {
        taskQueue = new ArrayBlockingQueue<>(maxNoOfTasks);

        for (int i = 0; i < noOfThreads; i++)
            runnables.add(new PoolThreadRunnable(taskQueue));

        for (PoolThreadRunnable runnable : runnables)
            new Thread(runnable).start();
    }

    public synchronized void execute(Runnable task) throws Exception {
        if (this.isStopped || !taskQueue.offer(task)) throw
                new IllegalStateException("ThreadPool is stopped");
    }

    public synchronized void stop() {
        this.isStopped = true;
        for (PoolThreadRunnable runnable : runnables) {
            runnable.doStop();
        }
    }

    public synchronized void waitUntilAllTasksFinished() {
        while (this.taskQueue.size() > 0) {
            try {
                Thread.sleep(1);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

In [ ]:

import java.util.concurrent.BlockingQueue;

public class PoolThreadRunnable implements Runnable {

    private Thread thread = null;
    private BlockingQueue<Runnable> taskQueue;
    private boolean isStopped = false;

    public PoolThreadRunnable(BlockingQueue<Runnable> queue) {
        taskQueue = queue;
    }

    public void run() {
        this.thread = Thread.currentThread();
        while (!isStopped()) {
            try {
                Thread.sleep(1000);
                Runnable runnable = taskQueue.take();
                runnable.run();
            } catch (Exception ignored) {}
        }
    }

    public synchronized void doStop() {
        isStopped = true;
        this.thread.interrupt();
    }

    public synchronized boolean isStopped() {
        return isStopped;
    }
}


In [ ]:
public class Main {
    public static void main() throws Exception {

        MyThreadPool threadPool = new MyThreadPool(3, 20);

        for (int i = 0; i < 20; i++) {

            int taskNumber = i;
            threadPool.execute(() -> {
                String message =
                        Thread.currentThread().getName()
                                + ": Task " + taskNumber;
                System.out.println(message);
            });
        }

        threadPool.waitUntilAllTasksFinished();
        threadPool.stop();

    }
}

## 12.4 `BlockingQueue`

![bq](bq.png)

## 12.5 `Lock`

![lock](lock.png)

In [ ]:
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

public class MyLock {
    private static final int COUNT_UP_TO = 1000;
    private static final int NUM_OF_THREADS = 100;

    private static final Lock lock = new ReentrantLock();

    private static int mCount = 0;

    public static void main(String[] args) throws InterruptedException {

        for(int i = 0; i < NUM_OF_THREADS; i++){
            startThread();
        }
        Thread.sleep(1000);
        System.out.println(mCount);
    }

    private static void startThread() {
        new Thread(() ->{
            for(int i = 0; i < COUNT_UP_TO; i++){
                lock.lock();
                mCount++;
                lock.unlock();
            }
        }).start();
    }
}

## 12.6 `Synchronized` vs `Lock`

- `synchronized` jest ograniczony do jednej metody
- `lock` i `unlock` mogą być wywołane w różnych metodach
- `synchronized` nie wspiera bezstronności
- `Lock` posiada właściwość `fairness` - sprawia że dostęp otrzymuje najdłużej oczekujący wątek
- `Lock` posiada metodę `tryLock`
- wątek oczekujący na dostęp do bloku `synchronized` nie może zostać przerwany